In [57]:
import pandas as pd
import numpy as np
import json

In [59]:
!conda install html5lib --yes
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f5704cfeade14e07882b78db987a8a2b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='teQJgdq6avpN8BGseiKmBgky-jZFY-Gx4ghixYKSFRCe',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_f5704cfeade14e07882b78db987a8a2b.get_object(Bucket='machinelearning-donotdelete-pr-vtqqcc9dbpor4i',Key='toronto.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


Solving environment: done

# All requested packages already installed.



,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [64]:
df=df_data_1[df_data_1.Borough != "Not assigned"]
df.head(20)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [70]:
df['Postcode'].nunique()

103

In [65]:
df['Neighbourhood']=df['Neighbourhood'].replace('Not assigned', df['Borough'])
df.head(20)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [76]:
df1=df.groupby(["Postcode","Borough"])["Neighbourhood"].apply(list)
df1=df1.sample(frac=1).reset_index()
df1["Neighbourhood"]=df1["Neighbourhood"].str.join(',')
df1.head(20)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M5X,Downtown Toronto,"First Canadian Place,Underground city"
2,M7A,Downtown Toronto,Queen's Park
3,M6C,York,Humewood-Cedarvale
4,M2J,North York,"Fairview,Henry Farm,Oriole"
5,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market"
6,M9P,Etobicoke,Westmount
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
8,M4M,East Toronto,Studio District
9,M3J,North York,"Northwood Park,York University"


In [77]:
df1.shape

(103, 3)

In [78]:
# Load coordinate data from CSV
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
# This assumes that the postal code column in your main dataframe is labelled 'PostalCode', this renames the column from the geo csv so the merge can happen automatically and cleanly
geo_df = geo_df.rename({'Postal Code':'Postcode'}, axis=1)

# Merge coordinates into neighbourhood dataframe
df1 = df1.merge(geo_df)
df1.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M5X,Downtown Toronto,"First Canadian Place,Underground city",43.648429,-79.382280
2,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
3,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
4,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
5,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049
6,M9P,Etobicoke,Westmount,43.696319,-79.532242
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
8,M4M,East Toronto,Studio District,43.659526,-79.340923
9,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
